In [ ]:
from custom_envs.dm_control_suite import get_default_config, load, get_domain_randomizer
from custom_envs.manipulation import get_default_config, load, get_domain_randomizer
import jax
import jax.numpy as jnp
import os
from learning.module.wrapper.adv_wrapper import wrap_for_adv_training
from learning.module.wrapper.dr_wrapper import wrap_for_dr_training
import functools
# os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]='false'
import mediapy as media


In [ ]:
env_name='PandaRobotiqPushCube'
cfg = get_default_config(env_name)
env = load(env_name, cfg)


In [ ]:
jit_step = jax.jit(env.step)
jit_reset = jax.jit(env.reset)


In [ ]:
init_state = jit_reset(jax.random.PRNGKey(2))
state = init_state


In [ ]:
seconds = 3
rollout=[state]
for i in range(int(seconds/env.sim_dt)):
    state = jit_step(state, jnp.ones(env.action_size))
    rollout.append(state)
frames = env.render(rollout)
media.show_video(frames, fps=1.0 / env.sim_dt)


In [ ]:
import mujoco
[
    (
        mujoco.mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_BODY, i), 
        state.data.xpos[i]
    ) 
    for i in range(env.mj_model.nbody)
]


In [ ]:
env.mjx_model.opt.gravity


In [ ]:
env.mjx_model.body_ipos


In [ ]:
state.data.geom_xpos


In [ ]:
env.mjx_model.geom_friction


In [ ]:
env.mjx_model.dof_frictionloss


In [ ]:
env.mjx_model.dof_armature


In [ ]:
env.mjx_model.dof_damping


In [ ]:
env.mjx_model.nv


In [ ]:
env.mjx_model.nu


In [ ]:
env.mjx_model.qpos0.shape


In [ ]:
env.mjx_model.actuator_gainprm


In [ ]:
env.mjx_model.actuator_biasprm


In [ ]:
from mujoco import mj_id2name, mj_name2id
import mujoco
[mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_BODY, i) for i in range(env.mj_model.nbody)]


In [ ]:

[
    (
        mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_JOINT, i),
        env.mj_model.dof_frictionloss[i]
    )
    for i in range(env.mj_model.njnt)]


In [ ]:
[
    (
        mujoco.mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_GEOM, i), 
        env.mj_model.geom_friction[i]
    ) 
    for i in range(env.mj_model.ngeom)
]


In [ ]:
env.mj_model.geom_friction.shape



In [ ]:
env._left_finger_geom


In [ ]:
[
    (
        mujoco.mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_BODY, i), 
        env.mj_model.body_mass[i]
    ) 
    for i in range(env.mj_model.nbody)
]


In [ ]:

init_state=  env.reset(jax.random.PRNGKey(0))
state=init_state



In [ ]:
from custom_envs import mjx_env


state.data
mjx_env.get_sensor_data(
        env.mj_model, state.data, "torso_subtreelinvel"
    )


In [ ]:
state = env.step(state, jnp.ones(env.action_size))
state.data.xpos



In [ ]:
from learning.module.wrapper.evaluator import AdvEvaluator


evaluator = AdvEvaluator(
        env,
        lambda x : x,
        num_eval_envs=128,
        episode_length=1000,
        action_repeat=2,
        key=jax.random.PRNGKey(0),
    )
dynamics_params = jnp.zeros((128,14))
metrics = evaluator.run_evaluation(dynamics_params, dynamics_params, {})
